# Test Your Algorithm

## Instructions
1. From the **Pulse Rate Algorithm** Notebook you can do one of the following:
   - Copy over all the **Code** section to the following Code block.
   - Download as a Python (`.py`) and copy the code to the following Code block.
2. In the bottom right, click the <span style="color:blue">Test Run</span> button. 

### Didn't Pass
If your code didn't pass the test, go back to the previous Concept or to your local setup and continue iterating on your algorithm and try to bring your training error down before testing again.

### Pass
If your code passes the test, complete the following! You **must** include a screenshot of your code and the Test being **Passed**. Here is what the starter filler code looks like when the test is run and should be similar. A passed test will include in the notebook a green outline plus a box with **Test passed:** and in the Results bar at the bottom the progress bar will be at 100% plus a checkmark with **All cells passed**.
![Example](example.png)

1. Take a screenshot of your code passing the test, make sure it is in the format `.png`. If not a `.png` image, you will have to edit the Markdown render the image after Step 3. Here is an example of what the `passed.png` would look like 
2. Upload the screenshot to the same folder or directory as this jupyter notebook.
3. Rename the screenshot to `passed.png` and it should show up below.
![Passed](passed.png)
4. Download this jupyter notebook as a `.pdf` file. 
5. Continue to Part 2 of the Project. 

In [2]:
import glob

import numpy as np
import scipy as sp
import scipy.io
import scipy.signal


def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl, fs = 125)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

def RunPulseRateAlgorithm(data_fl, ref_fl, fs = 125):
    """
    Makes pulse rate estimation and confidence estimation

    Args:
        data_fl: (str) filepath to a troika .mat file.
    Returns:
        errors: Calculated errors for each heart rate
        confidence: Calculated confidence for each heart rate
    """
    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    
    # Compute pulse rate estimates and estimation confidence.
    # Bandpass filter
    ppg, accx, accy, accz = bandpass_filter(ppg, fs), bandpass_filter(accx, fs), bandpass_filter(accy, fs), bandpass_filter(accz, fs)
    
    # Magnitude of acceleration as 3D
    acc = np.sqrt(accx**2 + accy**2 + accz**2)
    
    # Load ground truth as column vector and convert to row vector
    ground_truth = scipy.io.loadmat(ref_fl)['BPM0'].reshape(-1) 
    
    # Calculate heart rate prediction, confidence and errors
    predictions, confidence = predict_heart_rate(acc, ppg, fs)
    errors = np.abs(np.subtract(predictions, ground_truth))
    
    # Return per-estimate mean absolute error and confidence
    errors, confidence = np.array(errors), np.array(confidence)
    return errors, confidence

def bandpass_filter(signal, fs, pass_band=(40/60.0, 240/60.0)):
    """
    Filter bandpass for unrealistic heart rate values.
    Args:
        signal: numpy array. For ppm or acc signal
        fs: int. Sampling frequency in Hz
        pass_band: tuple. band width for filter
    Returns:
        predictions: heart rate predicitons 
        confidence: confidence estimates
    """
    b, a = scipy.signal.butter(3, pass_band, btype='bandpass', fs=fs) # 3: Order of the filter 
    return scipy.signal.filtfilt(b, a, signal)

def predict_heart_rate(acc, ppg, fs, window_length = 8, window_shift = 2, min_bandpass = 40/60.0, max_bandpass = 240/60.0):
    """
    Predict heart rate and calculate confidence using ppg and accelerometer data

    Args:
        acc: numpy array. Acceleration signal file
        ppg: numpy array. PPG signal file
        fs: int. Sampling frequency in Hz
        window_length_s: int. Window_length in seconds
        window_shift_s: int. Window shift in seconds

    Returns:
        predictions: list. Predicited heart rates
        confidence: list. Estimated confidences
    """
    # Define window shapes with given values to function
    window_length = window_length * fs
    window_shift = window_shift * fs
    
    # Create empty lists to store calculations
    bpm_predictions = []
    confidence = []
    
    # Make calculations with shifting windows
    for origin in range(0, len(ppg) - window_length + 1, window_shift):
        
        # Estimation for current window
        ppg_window = ppg[origin:origin+window_length]
        acc_window = acc[origin:origin+window_length]
        
        # Fourier transform
        ppg_freqs, ppg_fft = fourier_transform(ppg_window, fs)
        acc_freqs, acc_fft = fourier_transform(acc_window, fs)
        
        # Remove unwanted frequencies 
        ppg_fft[ppg_freqs <= min_bandpass] = 0.0
        ppg_fft[ppg_freqs >= max_bandpass] = 0.0
        acc_fft[acc_freqs <= min_bandpass] = 0.0
        acc_fft[acc_freqs >= max_bandpass] = 0.0
    
        # Pick frequency with largerst FFT coefficient
        ppg_freq = ppg_freqs[np.argmax(ppg_fft, axis=0)]  
        acc_freq = acc_freqs[np.argmax(acc_fft, axis=0)]
        
        freq = ppg_freq
        conf = confidence_calculator(ppg_freqs, ppg_fft, ppg_freq)
        
        # If the dominant accelerometer frequency is the same as the PPG, estimated heart rate is biased.
        # Make new estimation sor second suitable frequency.
        if abs(ppg_freq - acc_freq) == 0:
            
            # Calculate frequency with second largerst FFT
            sec_freq = ppg_freqs[np.argsort(ppg_fft, axis=0)[-2]]
            sec_conf = confidence_calculator(ppg_freqs, ppg_fft, sec_freq)
            
            # If new freq is larger, choose this as freq
            if sec_conf > conf:
                freq, conf = sec_freq, sec_conf
        
        bpm_predictions.append(freq * 60) # Convert to bpm (from Hz)
        confidence.append(conf)
        
    return bpm_predictions, confidence

def fourier_transform(signal, fs):
    """
    Calculate the Fast Fourier Transfrom of given sequence
    Args:
        signal: numpy array. For ppm or acc signal
        fs: int. Sampling frequency in Hz
    Returns:
        freqs: numpy array. Frequency bins 
        fft: float. Magnitude of FFT
    """ 
    n_samples = len(signal)  # No zero padding
    freqs = np.fft.rfftfreq(n_samples, 1/fs)
    fft = np.abs(np.fft.rfft(signal, n_samples))
    return freqs, fft

def confidence_calculator(freqs, fft, freq, window_half_width=1):
    """
    Calculate confidence by Fourier transformes data inputs.
    Args:
        freqs: numpy array. Frequency bins 
        fft: float. Magnitude of FFT
        freq: float. Frequence for calculation confidence.
    Returns:
        confidence: float.
    """ 
    window = (freqs > freq - window_half_width) & (freqs < freq + window_half_width)  
    confidence = np.sum(fft[window]) / np.sum(fft)
    return confidence